In [1]:
import torch
import torch.nn as nn
import sys
import os

In [2]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
path = "./runs/2_32_100_1e-3_1e-10_Adam_0_25_3class/"

In [4]:
decided_model = f"{path}/best-model.pt" #Changed to selected model

In [5]:
from model import ViTDino

In [6]:
model = ViTDino(out_dim=3)

Using cache found in /home/bahadir/.cache/torch/hub/facebookresearch_dino_main


In [7]:
model.load_state_dict(torch.load(decided_model))
model.to(device)
model.eval()

ViTDino(
  (model): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(8, 8), stride=(8, 8))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate=none)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (1): Block(
        (norm1): LayerNor

In [8]:
from data import create_splits
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

In [9]:
train_set, val_set, test_set = create_splits(dataset_type="5class")
set(test_set.img_labels)

Prev train example count: 791, cur train example count: 752
Image dir: 683, Image labels: 683
Image dir: 35, Image labels: 35
Image dir: 703, Image labels: 703


{0, 1, 2, 3, 4}

In [10]:
train_set, val_set, test_set = create_splits(dataset_type="5class")

#Conversion Part
conversion = lambda x: 1 if ((x>=1) & (x<4)) else 2 if (x==4) else 0
test_set.img_labels = [conversion(i) for i in test_set.img_labels]

Prev train example count: 791, cur train example count: 752
Image dir: 681, Image labels: 681
Image dir: 31, Image labels: 31
Image dir: 703, Image labels: 703


In [11]:
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=2)

In [12]:
np.unique(np.array(test_set.img_labels))

array([0, 1, 2])

In [13]:
def test(device, model, criterion, test_loader, savedir):
    cur_test_loss = 0
    test_tot, test_corr = 0, 0
    model.eval()

    all_preds, all_labels, all_indices = [], [], []
    with torch.no_grad():
        for images, labels, index in test_loader:
            images, labels = images.to(device), labels.to(device)
            outs = model(images)
            test_loss = criterion(outs, labels)
            cur_test_loss += test_loss.item()

            _, preds = torch.max(outs.data, 1)
            test_tot += labels.size(0)
            test_corr += (preds == labels).sum().item()

            preds = preds.cpu().tolist()
            labels = labels.cpu().tolist()
            index = index.tolist()

            all_preds.extend(preds)
            all_labels.extend(labels)
            all_indices.extend(index)
            
    test_loss = cur_test_loss / len(test_loader)
    test_acc = test_corr / test_tot

    print(f'Test loss: {test_loss:.3f}, Test accuracy: {test_acc:.3f}')

    #Print predictions by class
    confusion_matrix = pd.crosstab(np.array(all_preds), np.array(all_labels), rownames=["Predictions"], colnames=["Actuals"]) 
    print(confusion_matrix)

    #Accuracy Results
    classification_res = classification_report(all_labels, all_preds, digits=3)
    print(classification_res)


In [14]:
test(device, model, nn.CrossEntropyLoss(), test_loader, None)

Test loss: 0.100, Test accuracy: 0.977
Actuals        0    1    2
Predictions               
0            177    0    0
1              0  409    4
2              1   11  101
              precision    recall  f1-score   support

           0      1.000     0.994     0.997       178
           1      0.990     0.974     0.982       420
           2      0.894     0.962     0.927       105

    accuracy                          0.977       703
   macro avg      0.961     0.977     0.969       703
weighted avg      0.978     0.977     0.978       703

